In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.datasets import mnist
mnist=mnist.load_data()

In [3]:
(train_images,train_labels),(test_images,test_labels)=mnist

In [4]:
train_images,test_images=train_images/255.0,test_images/255.0

In [5]:
train_images.shape

(60000, 28, 28)

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
from tensorflow.keras.utils import to_categorical
num_classes=10
train_labels=to_categorical(train_labels,num_classes)
test_labels=to_categorical(test_labels,num_classes)

In [8]:
input_width=28
input_height=28
input_channels=1
input_pixels=784

n_conv1=32
n_conv2=64
stride_conv1=1
stride_conv2=1
conv1_k=5
conv2_k=5
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_out=10

input_size_to_hidden=(input_width//(max_pool1_k*max_pool2_k))*(input_height//(max_pool1_k*max_pool2_k))*n_conv2

In [9]:
weights={
    "wc1":tf.Variable(tf.random.normal([conv1_k,conv1_k,input_channels,n_conv1])),
     "wc2":tf.Variable(tf.random.normal([conv2_k,conv2_k,n_conv1,n_conv2])),
     "wh1":tf.Variable(tf.random.normal([input_size_to_hidden,n_hidden])),
     "wo":tf.Variable(tf.random.normal([n_hidden,n_out]))
}
biases={
    "bc1":tf.Variable(tf.random.normal([n_conv1])),
    "bc2":tf.Variable(tf.random.normal([n_conv2])),
    "bh1":tf.Variable(tf.random.normal([n_hidden])),
    "bo":tf.Variable(tf.random.normal([n_out])),
}

In [10]:
def conv(x,weights,bias,strides=1):
    out=tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out

def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])

In [11]:
def cnn(x,weights,biases):
    x=tf.reshape(x,shape=[-1,input_height,input_width,input_channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool=maxpooling(conv1,max_pool1_k)
    
    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool=maxpooling(conv2,max_pool2_k)
    
    hidden_input=tf.reshape(conv2_pool,shape=[-1,input_size_to_hidden])
    hidden_output_before_activation=tf.add(tf.linalg.matmul(hidden_input,weights['wh1'],biases['bh1']))
    hidden_output=tf.nn.relu(hidden_output_before_activation)
    
    output=tf.add(tf.matmul(hidden_output,weights['w0']),biases['bo'])
    return output

In [12]:
x = tf.keras.layers.Input(shape=(input_pixels,))
y = tf.keras.layers.Input(shape=(n_out,))
pred=cnn(x,weights,biases)

OperatorNotAllowedInGraphError: Exception encountered when calling layer "tf.linalg.matmul" (type TFOpLambda).

Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.

Call arguments received by layer "tf.linalg.matmul" (type TFOpLambda):
  • a=tf.Tensor(shape=(None, 3136), dtype=float32)
  • b=<tf.Variable 'Variable:0' shape=(3136, 1024) dtype=float32>
  • transpose_a=<tf.Variable 'Variable:0' shape=(1024,) dtype=float32>
  • transpose_b=False
  • adjoint_a=F